<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [2]:
%cd /content/gdrive/MyDrive/HatefulMemesDetection/Code/Experiments

/content/gdrive/MyDrive/HatefulMemesDetection/Code/Experiments


In [3]:

# Check GPU availability
!nvidia-smi

Wed Apr 26 22:27:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!sudo apt-get update -y
!sudo apt-get install python3.7

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,150 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,341 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,334 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/main a

In [5]:
!pip install torch transformers torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.2 MB/s eta 0:00:00


In [6]:
'''Importing necessary modules
'''
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/Trainers/')
sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/Dataloaders/')
sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/utils/')

from dataloader import mycapdataset#, create_prime_dict 

'''
For BERT
'''
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from trainer_BERT import train, test_classify

In [7]:
# Select gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


**Dataloading Scheme**

In [8]:
trainlist = '/content/gdrive/MyDrive/hm_project_pe/data/Generated_Caps/train_cap.jsonl'
validlist = '/content/gdrive/MyDrive/hm_project_pe/data/Generated_Caps/dev_cap.jsonl'
img_dir = '/content/gdrive/MyDrive/hm_project_pe/data'

In [9]:
# prime_dict = create_prime_dict(trainlist)

In [10]:
'''
Train Dataloader
''' 
train_dataset = mycapdataset(annotations_file=trainlist, img_dir=img_dir, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 32, num_workers=16,pin_memory=True)


'''
Validation Dataloader
''' 
validation_dataset = mycapdataset(annotations_file=validlist, img_dir=img_dir, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 32, num_workers=16,pin_memory=True)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_ms

**Model Definition**

In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [12]:
'''
Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", #12-layer BERT model, with an uncased vocab.
    num_labels = 2, #Binary classification
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model = nn.DataParallel(model,device_ids=[0]).to(device)
model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, 

**Hyperparameters**

In [13]:
'''
Loss Function
'''
criterion = nn.CrossEntropyLoss()

lr_list = [2e-3, 2e-4, 2e-5]

for lr in lr_list:

  '''
  Optimizer
  '''
  optimizer = AdamW(model.parameters(), lr = lr, eps = 1e-8)

  '''
  Number of training epochs. The BERT authors recommend between 2 and 4. We chose to run for 4, but we'll see later that this may be over-fitting the training data.
  '''
  num_Epochs = 4 #changed from 10


  '''
  Create the learning rate scheduler.
  Total number of training steps is [number of batches] x [number of epochs].
  Note that this is not the same as the number of training samples.
  '''
  total_steps = len(train_dataloader) * num_Epochs
  lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,  num_training_steps = total_steps)

  modelname = 'BERT_Gen_Cap_basic'
  modelpath = './saved_model_checkpoints/'+modelname

  writer = SummaryWriter(modelname)

  train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)

  writer.flush()
  writer.close()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:  1
training loss =  0.7109182833280778
Validation Loss: 0.7031	Top 1 Validation Accuracy: 0.5000
Epoch:  2
training loss =  0.674840689489716
Validation Loss: 0.9724	Top 1 Validation Accuracy: 0.5000
Epoch:  3
training loss =  0.6780412498497426
Validation Loss: 0.6961	Top 1 Validation Accuracy: 0.5000
Epoch:  4
training loss =  0.6595071424218945
Validation Loss: 0.7420	Top 1 Validation Accuracy: 0.5000
Epoch:  1
training loss =  0.6598756913851974
Validation Loss: 0.7299	Top 1 Validation Accuracy: 0.5000
Epoch:  2
training loss =  0.661115818229833
Validation Loss: 0.7314	Top 1 Validation Accuracy: 0.5000
Epoch:  3
training loss =  0.6580059079075218
Validation Loss: 0.7162	Top 1 Validation Accuracy: 0.5000
Epoch:  4
training loss =  0.6582557928741426
Validation Loss: 0.7351	Top 1 Validation Accuracy: 0.5000
Epoch:  1
training loss =  0.6562198918116721
Validation Loss: 0.7349	Top 1 Validation Accuracy: 0.5000
Epoch:  2
training loss =  0.6557902942474624
Validation Loss: 0.7

In [14]:
'''
Load saved model from checkpoint  #####
'''
from Load_model import load
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)


In [ ]:
model.module.classifier

In [ ]:
epoch

In [ ]:
v_loss

In [ ]:
train_loss

In [ ]:
v_acc

In [ ]:
test_loss, accuracy, original_label, predicted_label, prediction_probs = test_classify(model, validation_dataloader, criterion, device)

In [ ]:
import pandas as pd
original = pd.read_json("/content/gdrive/MyDrive/HM_Dev_Results/dev_original.jsonl", lines=True)

In [ ]:
original.head()

,id,img,label,text
0,8291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...
4,80243,img/80243.png,1,mississippi wind chime


In [ ]:
original['original_label'] = original_label
original['predicted_label'] = predicted_label
original['prediction_probs'] = prediction_probs

In [ ]:
original.head()

,id,img,label,text,original_label,predicted_label,prediction_probs
0,8291,img/08291.png,1,white people is this a shooting range,1,0,0.782657
1,46971,img/46971.png,1,bravery at its finest,1,0,0.954585
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,1,0,0.722084
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,1,0,0.861270
4,80243,img/80243.png,1,mississippi wind chime,1,0,0.534593


In [ ]:
original.to_csv('BERT_Cap_Gen_prediction.csv', index=False)